# 大模型调用外部百度浏览器（API）实现在线搜索并回答

In [10]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Type
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field, PrivateAttr
from langchain_core.tools import BaseTool
import requests
import sys

In [4]:
class BaiduInput(BaseModel):
    query: str = Field(..., description="搜索的查询内容")

class BaiduSearch(BaseTool):
    name: str = "baidu_web_search"
    description: str = "使用百度进行搜索"
    args_schema: Type[BaseModel] = BaiduInput

    _api_key: str = PrivateAttr()
    _count: int = PrivateAttr()
    _summary: bool = PrivateAttr()
    _freshness: str = PrivateAttr()

    def __init__(self, api_key: str, count: int = 5, summary: bool = True, freshness: str = "noLimit", **kwargs):
        super().__init__(**kwargs)
        self._api_key = api_key
        self._count = count
        self._summary = summary
        self._freshness = freshness

    def _run(self, query: str) -> str:
        url = "https://qianfan.baidubce.com/v2/ai_search"
        headers = {
            "Authorization": f"Bearer {self._api_key}",
            "Content-Type": "application/json"
        }

        message = [
            {
                "content": query,
                "role": "user"
            }
        ]

        data = {
            "messages": message,
            "search_source": "baidu_search_v2",

        }

        try:
            response = requests.post(url, headers=headers, json=data, timeout=10)
            response.raise_for_status()
            data = response.json() # data 是返回的JSON数据 (字典)

            results = data.get("references", {}) # results 是一个 1 维列表，包含若干个搜索结果（字典）
            # print(type(results))
            # print("-----"*40)
            # print(results)
            # print("-----"*40)
            # print(len(results))
            # print("-----"*40)
            # print(results[0])
            # print("-----"*40)
            if not results:
                return "未找到相关内容。\n[DEBUG] 返回数据：{}".format(data)

            output = ""
            for i, item in enumerate(results[:self._count]):
                title = item.get("title", "无标题")
                snippet = item.get("content", "无摘要")
                url = item.get("url", "")
                output += f"{i+1}. {title}\n{snippet}\n链接: {url}\n\n"

            return output.strip()

        except Exception as e:
            return "搜索失败: {}".format(e)

In [ ]:
Search = BaiduSearch(api_key="", count=4)

result = Search.invoke({"query": "阿里巴巴2024年的ESG报告"})
print(result)

1. ESG信披案例丨阿里巴巴设立三级ESG治理架构 创造性提出“范围3+”新概念
 9月1日,阿里巴巴(09988.HK,股价137.10港元,市值2.61万亿港元)开盘大涨,截至收盘涨幅达18.5%。在消息面上,有报道称阿里巴巴开发了一款新的AI(人工智能)芯片,以填补英伟达在中国市场的空白。这款新芯片目前正在测试中,旨在服务于更广泛的人工智能推理任务,而且与英伟达兼容。 今年6月底,阿里巴巴发布了2025环境、社会和治理(ESG)报告(以下简称“报告”),这是其连续第四年发布ESG报告,共计131页,分为治理、环境和社会三大部分。报告显示,阿里巴巴设立三级ESG治理架构,还创造性提出“范围3+”新概念。 中央财经大学绿色金融国际研究院副院长施懿宸接受《每日经济新闻》记者(以下简称“每经记者”)微信采访时表示:“通过将‘范围 3+’纳入治理与考核体系,阿里巴巴既厘清自身碳责任边界,又以平台杠杆放大减排效应,为全球电商行业提供了可复制的价值链治理范式。” 建立三级ESG治理架构 最看重产品与服务质量 报告显示,2025财年阿里巴巴识别了21项关键议题,其中“产品与服务质量”“科技创新”“私隐保护与数据安全”三大议题的双重重要性最高。 图片来源:阿里巴巴2025环境、社会和治理(ESG)报告 报告再次强调了公司愿景:“追求成为一家活102年的好公司,我们的文化、商业模式和系统的建立都要经得起时间考验。” 2024年9月10日,在阿里巴巴成立25周年之际,马云也曾在阿里巴巴内网发帖鼓励阿里人说:“阿里之所以是阿里,那是因为我们有理想主义精神,我们相信未来,我们相信市场,我们更加相信只有为社会创造真正价值的善良公司才能坚持走完102年路程。” 在篇首集团首席执行官的信中,吴泳铭坦言:“2025年,全球面临的可持续发展挑战愈发艰巨。但自从四年前确立ESG目标以来,公司持续推进ESG行动的决心没有变,不断完善内部组织与治理机制没有变,ESG依然是一把重要标尺,度量和校准我们发展的品质。” 阿里巴巴直接从管理层面加强ESG实践,建立了三级ESG治理架构。该架构由可持续发展委员会(SC)、可持续发展管理委员会(SSC)和 ESG 工作组组成,工作内容分别是监督和评估ESG相关计划及项目的实施与成效、负责制定并落实ESG战略与目标及负责日常ESG相关工作的具体执行。 施懿宸认为,

In [5]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent():
    def __init__(self, model, tools, system = ""):
        self.system = system
        graph = StateGraph(AgentState) # 创建状态图，并且定义每个状态的类型 （每个状态是个字典，包含 messages 键，键的对应值是列表）

# ----------------------------------------------------------- 添加节点 ----------------------------------------------------------

        graph.add_node("llm", self.call_openai)

        graph.add_node("action", self.take_action)

        # 添加条件跳转逻辑：
        # 如果 llm 的输出中包含 tool_calls，就跳转到 action 节点执行；
        # 否则直接结束流程（END）
        graph.add_conditional_edges(
            "llm",
            self.exists_action, # 这个函数返回 True 或 False
            {True: "action", False: END}
        )

        # 执行完 action 节点后，再回到 llm 节点继续处理对话
        graph.add_edge("action", "llm")   # graph.add_edge("前一节点", "后一节点")

        graph.set_entry_point("llm") # 设置开始节点
        self.graph = graph.compile()    

#----------------------------------------------------------- 接入工具和模型 ----------------------------------------------------------

        self.tools = {t.name: t for t in tools}


        '''
        tool一般作为列表传进
        tools = [
            Tool(name="search_weather", func=search_weather_func),
            Tool(name="calculate", func=calculator_func)
        ]
        所以输出就是 tools = { "search_weather": Tool(...), "calculate": Tool(...) }
        '''
        
        self.model = model.bind_tools(tools)  # 把工具接入模型

# ----------------------------------------------------------- 模型调用函数 ----------------------------------------------------------
# 根据当前对话状态，调用语言模型并返回回复消息。

    def call_openai(self, state: AgentState):
        messages = state['messages']  # 取出字典state里的 messages 键对应的值（列表）
        print("--------"*20)
        print("\n")
        print("message: #########{}".format(messages))
        print("\n")
        print("--------"*20)
        print("\n")
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages  # 把system提示词封装成SystemMessage对象，并插入到 messages 列表的最前面
        message = self.model.invoke(messages)   # LangChain 的 .invoke() 调用模型
        return {'messages': [message]} # 返回AIMessage格式，符合自定义的AgentState里的 messages 键的类型要求
    
# ----------------------------------------------------------- 工具使用判定 ----------------------------------------------------------

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]   # 看最后一条传入的信息
        print("--------"*20)
        print("\n")
        print("result in exist action: #########",result)
        print("\n")
        print("--------"*20)
        print("\n")
        return len(result.tool_calls) > 0   #用 .tool_calls的方法去看到底有几条方法可以调用，返回的是列表里嵌套字典。 如果是0则不需要调用就返回False，否则是True
    
    """
    返回的 tool_calss 是个列表，里面是字典
    AIMessage(
    content="", 
    tool_calls=[
    {
            "id": "tool_call_1",
            "name": "search_weather",
            "args": {"city": "北京"}
             }
    ],
)
    """

# ----------------------------------------------------------- 执行工具调用 ----------------------------------------------------------

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        print("--------"*20)
        print("\n")
        print("tool calls: #########{}".format(tool_calls))
        print("\n")
        print("--------"*20)
        print("\n")
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t['name']].invoke(t['args'])  # 工具的返回结果
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result))) # # 将执行结果打包成一个 ToolMessage（LangChain 约定格式），用于反馈给模型
        print("--------"*20)
        print("\n")
        print("tool results: #########{}".format(results))
        print("\n")
        print("--------"*20)
        print("\n")
        print("Back to the model!")
        return {'messages': results}  # 返回格式为一个包含 ToolMessage 列表的 dict，LangGraph 会自动加进 state['messages']
    
    """
    调用查找工具 search_weather,执行 search_weather("北京")  返回 "北京 今天晴,气温26℃
    包装成
    sToolMessage(
        tool_call_id="tool_call_1",
        name="search_weather",
        content="北京 今天晴,气温26℃"
    )
    """

In [6]:
prompt = """你是一名聪明的科研助理。可以使用搜索引擎查找信息。
你可以多次调用搜索（可以一次性调用，也可以分步骤调用）。
只有当你确切知道要找什么时才去检索信息。
如果在提出后续问题之前需要先检索信息，你也可以这样做！
返回的答案请包括你引用文章对应的链接（如果链接存在的话）.
"""

In [ ]:
model = ChatOpenAI(
    model="gemini-2.5-flash",
    api_key="",
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
tool = BaiduSearch(api_key="", count=10)
agent = Agent(model, [tool], system=prompt)

In [13]:
ascii_diagram = agent.graph.get_graph().draw_ascii()
print(ascii_diagram)

       +-----------+          
       | __start__ |          
       +-----------+          
              *               
              *               
              *               
          +-----+             
          | llm |             
          +-----+.            
         .        .           
       ..          ..         
      .              .        
+--------+       +---------+  
| action |       | __end__ |  
+--------+       +---------+  


In [14]:
# ----------------------------- 测试使用 -----------------------------
# 如果要在终端print，注释掉带sys的四行

original_stdout = sys.stdout  # 复制备份原始的输出模式
sys.stdout = open("output.txt", "w")  # 将输出定向到指定文件。 w 覆盖模式，续写用 a
messages = [HumanMessage(content="香港城市大学2026年研究生招生信息，人工智能，数据科学方向")]
result = agent.graph.invoke({"messages": messages})
print(result['messages'][-1].content)
sys.stdout.close()  # 关闭文件
sys.stdout = original_stdout  # 恢复原始的标准输出